In [40]:
# pandas:
# - Used to load CSV files
# - Used to clean, merge, and manipulate patient data tables
# - Used for feature engineering (handling missing values, encoding, etc.)
import pandas as pd


# numpy:
# - Used for numerical operations
# - Used to create sample weights (event vs censored)
# - Used for mathematical calculations like RMSE
import numpy as np


# datetime:
# - Used to work with dates and times
# - Helps calculate survival time (diagnosis → death / last visit)
# - Ensures correct date arithmetic
from datetime import datetime


# GradientBoostingRegressor:
# - Machine learning model used to predict survival time
# - Builds multiple decision trees sequentially
# - Captures non-linear relationships and feature interactions
from sklearn.ensemble import GradientBoostingRegressor


# train_test_split:
# - Splits data into training and testing sets
# - Ensures unbiased model evaluation
#
# cross_val_score:
# - Used for cross-validation (optional improvement step)
# - Helps estimate model stability across multiple data splits
from sklearn.model_selection import train_test_split, cross_val_score


# concordance_index:
# - Evaluation metric specific to survival analysis
# - Measures how well the model ranks patients by survival time
# - Properly handles censored data
from lifelines.utils import concordance_index


# matplotlib.pyplot:
# - Used for plotting and visualization
# - Can be used for:
#   • Feature importance plots
#   • Predicted vs actual survival curves
#   • Risk group visualizatio


In [2]:
print("="*70)
print("GRADIENT BOOSTING SURVIVAL MODEL")
print("="*70)
print("  ✓ Tree-based machine learning model")
print("  ✓ Captures non-linear relationships automatically")
print("  ✓ Handles feature interactions")


GRADIENT BOOSTING SURVIVAL MODEL

Gradient Boosting as alternative to Cox Proportional Hazards:
  ✓ Tree-based machine learning model
  ✓ No proportional hazards assumption needed
  ✓ Captures non-linear relationships automatically
  ✓ Handles feature interactions
  ✓ Uses sklearn (no extra dependencies)


In [3]:
base_path = r"C:\Users\sanskar.kashyap\OneDrive - Mu Sigma Business Solutions Pvt. Ltd\Desktop\Model-BMS"
df_nsclc = pd.read_csv(f'{base_path}\\nscexpnd_nsclc_2506.csv')
df_mortality = pd.read_csv(f'{base_path}\\nscexpnd_mortality_v2_2506.csv')
df_demographics = pd.read_csv(f'{base_path}\\nscexpnd_demographics_2506.csv')
df_ecog = pd.read_csv(f'{base_path}\\nscexpnd_ecog_2506.csv')
df_visits = pd.read_csv(f'{base_path}\\nscexpnd_visit_2506.csv')

print("\n✓ All datasets loaded successfully")


✓ All datasets loaded successfully


In [22]:
# 1️⃣ Keep ONLY NSCLC patients
# Why?
# - Our study is about lung cancer (NSCLC)
# - Other cancer patients would confuse the model
cohort = df_nsclc[df_nsclc["isnsclc"] == 1].copy()


# 2️⃣ Define the start of survival time
# Why?
# - Survival time always starts from diagnosis date
# - All patients must have a common "time zero"
cohort["start_date"] = pd.to_datetime(cohort["nsclcdiagnosisdate"])


# 3️⃣ Select only required mortality columns
# Why?
# - We only need patient ID and death date
# - Reduces memory usage and keeps data clean
mort = df_mortality[["patientid", "dateofdeath"]].copy()


# 4️⃣ Convert death date to datetime format
# Why?
# - Date calculations require proper datetime type
# - Strings cannot be used for date subtraction
mort["dateofdeath"] = pd.to_datetime(mort["dateofdeath"])


# 5️⃣ Merge death information with patient cohort
# Why?
# - Adds death information to each patient
# - LEFT join keeps all patients (alive + dead)
cohort = cohort.merge(mort, on="patientid", how="left")


# 6️⃣ Create event indicator (core survival concept)
# Why?
# - event = 1 → patient died (observed outcome)
# - event = 0 → patient alive (censored)
# - Survival models NEED this information
cohort["event"] = cohort["dateofdeath"].notna().astype(int)


# 7️⃣ Find the last hospital visit for each patient
# Why?
# - For alive patients, we only know survival until last visit
# - This defines the censoring time
last_visit = df_visits.groupby("patientid")["visitdate"].max().reset_index()


# 8️⃣ Convert visit date to datetime
# Why?
# - Required for accurate time calculations
last_visit["visitdate"] = pd.to_datetime(last_visit["visitdate"])


# 9️⃣ Merge last visit date into cohort
# Why?
# - Adds follow-up information for censored patients
cohort = cohort.merge(last_visit, on="patientid", how="left")


# 🔟 Define study end date (data cutoff)
# Why?
# - Prevents survival time from extending beyond data availability
# - Standard practice in clinical studies
DATA_CUTOFF = pd.to_datetime("2025-01-01")


# 1️⃣1️⃣ Initialize end_date using date of death
# Why?
# - For patients who died, survival ends at death
cohort["end_date"] = cohort["dateofdeath"]


# 1️⃣2️⃣ For alive patients, use last visit date as end_date
# Why?
# - We don't know when they will die
# - We only know they survived until last visit
cohort.loc[cohort["event"] == 0, "end_date"] = \
    cohort.loc[cohort["event"] == 0, "visitdate"]


# 1️⃣3️⃣ Handle patients with no death and no visit record
# Why?
# - Avoid missing end dates
# - Assume survival until study cutoff
cohort["end_date"] = cohort["end_date"].fillna(DATA_CUTOFF)


# 1️⃣4️⃣ Calculate overall survival time (OS)
# Why?
# - This is the TARGET variable for the model
# - Measures how long the patient survived after diagnosis
cohort["os_time_days"] = (cohort["end_date"] - cohort["start_date"]).dt.days


# 1️⃣5️⃣ Ensure survival time is numeric
# Why?
# - Prevents errors during model training
# - Converts invalid values to NaN
cohort["os_time_days"] = pd.to_numeric(cohort["os_time_days"], errors="coerce")


# 1️⃣6️⃣ Identify invalid survival times
# Why?
# - Survival cannot be negative or zero
# - Missing values break survival analysis
invalid_rows = cohort["os_time_days"].isna() | (cohort["os_time_days"] <= 0)


# 1️⃣7️⃣ Log how many rows are removed
# Why?
# - Transparency and data quality check
print(f"✓ Dropped {invalid_rows.sum()} invalid OS rows")


# 1️⃣8️⃣ Remove invalid rows from the cohort
# Why?
# - Ensures clean, meaningful data for modeling
# - Improves model reliability
cohort = cohort.loc[~invalid_rows].copy()


✓ Dropped 61 invalid OS rows


In [23]:
# --- C. Merge Demographics & ECOG ---

# 1️⃣ Merge demographic information into the cohort
# Why?
# - Adds basic patient characteristics needed for survival prediction
# - Age, sex, and race are known prognostic factors
# - LEFT join keeps all patients even if demographic info is missing
cohort = cohort.merge(
    df_demographics[["patientid", "birthyear", "birthsex", "race"]],
    on="patientid",
    how="left"
)


# 2️⃣ Calculate patient's age at diagnosis
# Why?
# - Age at diagnosis affects survival, not current age
# - Survival models need age as a numerical feature
cohort["age"] = cohort["start_date"].dt.year - cohort["birthyear"]


# 3️⃣ Create a working copy of ECOG data
# Why?
# - Avoids modifying the original ECOG dataset
ecog = df_ecog.copy()


# 4️⃣ Convert ECOG date to datetime format
# Why?
# - Required to compare ECOG date with diagnosis date
# - String dates cannot be compared reliably
ecog["ecogdate"] = pd.to_datetime(ecog["ecogdate"])


# 5️⃣ Attach diagnosis date to each ECOG record
# Why?
# - Allows filtering ECOG values taken BEFORE diagnosis
# - Prevents using post-diagnosis information (data leakage)
ecog = ecog.merge(
    cohort[["patientid", "start_date"]],
    on="patientid",
    how="inner"
)


# 6️⃣ Keep only ECOG measurements taken on or before diagnosis
# Why?
# - ECOG after diagnosis may be influenced by disease progression
# - Using post-diagnosis values would leak future information
ecog = ecog[ecog["ecogdate"] <= ecog["start_date"]]


# 7️⃣ Select the most recent ECOG BEFORE diagnosis (baseline ECOG)
# Why?
# - Closest ECOG before diagnosis best represents patient condition at baseline
# - Group by patient and take the latest valid ECOG
baseline_ecog = (
    ecog
    .sort_values("ecogdate")
    .groupby("patientid")
    .last()
    .reset_index()
)


# 8️⃣ Merge baseline ECOG into main cohort
# Why?
# - Adds functional status as an important survival predictor
# - LEFT join keeps patients even if ECOG is missing
cohort = cohort.merge(
    baseline_ecog[["patientid", "ecogvalue"]],
    on="patientid",
    how="left"
)


In [24]:
# --- D. Prepare Modeling Dataset ---

# 1️⃣ Select only the required columns for survival modeling
# Why?
# - Keeps the dataset clean and focused
# - Removes unused columns that could confuse the model
# - Each selected column has clinical or predictive importance
os_df = cohort[[
    "patientid",        # Unique patient identifier (not used for modeling, only tracking)
    "os_time_days",     # Target variable: overall survival time in days
    "event",            # Event indicator: 1 = death, 0 = censored (alive)
    "age",              # Age at diagnosis (strong survival predictor)
    "birthsex",         # Biological sex (may affect outcomes)
    "race",             # Race (can capture demographic outcome differences)
    "ecogvalue",        # Baseline functional status of patient
    "groupstage",       # Cancer stage at diagnosis (severity indicator)
    "ismetastatic",     # Whether cancer has spread to distant organs
    "histology",        # Cancer subtype (biological behavior differs)
    "smokingstatus",    # Smoking history (risk and prognosis factor)
    "hassurgery"        # Whether patient underwent surgery (treatment effect)
]].copy()


# 2️⃣ Print total number of patients in final dataset
# Why?
# - Sanity check to ensure no unexpected data loss
# - Confirms cohort size before modeling
print(f"\n✓ Final cohort: {len(os_df)} patients")


# 3️⃣ Print number of death events
# Why?
# - Shows how many patients have observed outcomes
# - Important for survival model reliability
print(f"  - Events (deaths): {os_df['event'].sum()}")


# 4️⃣ Print number of censored patients
# Why?
# - Indicates how many patients are still alive or lost to follow-up
# - Survival models must handle censoring properly
print(f"  - Censored: {(os_df['event'] == 0).sum()}")



✓ Final cohort: 1289 patients
  - Events (deaths): 708
  - Censored: 581


In [25]:
# --- E. Feature Engineering ---

# 1️⃣ Handle missing ECOG values
# Why?
# - Some patients do not have ECOG recorded
# - ECOG is an important predictor of survival
# - Dropping these patients would reduce sample size
# - Median is used because it is robust to extreme values
os_df["ecogvalue"] = os_df["ecogvalue"].fillna(
    os_df["ecogvalue"].median()
)


# 2️⃣ Handle missing surgery information
# Why?
# - Missing surgery usually means surgery was not performed
# - Converting to 0/1 makes it usable for ML models
# - Ensures consistent data type
os_df["hassurgery"] = (
    os_df["hassurgery"]
    .fillna(0)          # Assume no surgery if missing
    .astype(int)        # Convert to binary integer (0 or 1)
)


# 3️⃣ Handle missing age values
# Why?
# - Age is essential for survival prediction
# - Missing age cannot be left as NaN
# - Median preserves population structure better than mean
os_df["age"] = os_df["age"].fillna(
    os_df["age"].median()
)


In [26]:
# Use same dummy encoding as Cox for fair comparison
# Why?
# - Cox model cannot handle text (categorical) variables directly
# - Gradient Boosting also works better with numeric inputs
# - Using the SAME encoding ensures fair performance comparison
cat_cols = [
    "birthsex",        # Male / Female
    "race",            # White / Black / Asian / etc.
    "groupstage",      # Cancer stage (I, II, III, IV)
    "histology",       # Cancer subtype
    "smokingstatus"    # Smoking history
]


# Convert categorical (text) columns into numeric dummy variables
# Why?
# - Machine learning models cannot understand text labels
# - Each category is converted into a 0/1 column
# - This process is called One-Hot Encoding
os_df_encoded = pd.get_dummies(
    os_df,
    columns=cat_cols,   # Columns to be encoded
    drop_first=True     # Drop one category to avoid redundancy
)


# Remove patient identifier from modeling data
# Why?
# - patientid has no predictive meaning
# - Keeping it could cause data leakage or overfitting
# - It is only useful for tracking, not learning
model_df = os_df_encoded.drop(columns=["patientid"])


In [27]:
# --- F. Train-Test Split ---

# 1️⃣ Split the data into training and testing sets
# Why?
# - Training data is used to teach the model patterns
# - Testing data is used to check if the model works on new patients
# - Prevents the model from memorizing instead of learning
train_df, test_df = train_test_split(
    model_df,              # Final dataset after encoding and cleaning
    test_size=0.2,         # 20% data for testing, 80% for training
    random_state=42,       # Ensures the split is reproducible
    stratify=model_df["event"]  # Keeps same death/alive ratio in both sets
)


# 2️⃣ Print training set details
# Why?
# - Confirms how many patients are used to train the model
# - Shows number of death events available for learning
print(f"\n✓ Train set: {len(train_df)} patients ({train_df['event'].sum()} events)")


# 3️⃣ Print test set details
# Why?
# - Confirms how many patients are used for evaluation
# - Ensures enough death events exist for fair testing
print(f"✓ Test set: {len(test_df)} patients ({test_df['event'].sum()} events)")



✓ Train set: 1031 patients (566 events)
✓ Test set: 258 patients (142 events)


In [28]:
# --- G. Train Gradient Boosting Model ---

# 1️⃣ Print section header (for readability only)
# Why?
# - Makes output easier to read
# - Helps track progress during long runs
print("\n" + "=" * 70)
print("TRAINING GRADIENT BOOSTING MODEL")
print("=" * 70)


# 2️⃣ Prepare training INPUT features (X)
# Why?
# - X contains only patient characteristics
# - Model uses X to learn patterns
# - Survival time and event must NOT be included as inputs
X_train = train_df.drop(
    columns=[
        "os_time_days",  # Target variable (what we want to predict)
        "event"          # Outcome indicator (used only for evaluation/weighting)
    ]
)


# 3️⃣ Prepare training TARGET variable (y)
# Why?
# - y is what the model tries to predict
# - Here: overall survival time in days
y_train = train_df["os_time_days"]


# 4️⃣ Prepare testing INPUT features (X)
# Why?
# - Used to test model performance on unseen patients
X_test = test_df.drop(
    columns=[
        "os_time_days",
        "event"
    ]
)


# 5️⃣ Prepare testing TARGET variable (y)
# Why?
# - Used only for evaluation
# - Model NEVER sees these values during training
y_test = test_df["os_time_days"]



TRAINING GRADIENT BOOSTING MODEL


In [29]:
# Create sample weights: events get full weight, censored get partial weight
# Why?
# - Patients who died have exact survival time
# - Censored patients have incomplete survival information
# - We want the model to trust death events more
train_weights = np.where(
    train_df["event"] == 1,    # If patient died
    1.0,                       # Full importance
    0.5                        # Lower importance for censored patients
)


# Print training information (for clarity only)
print("\nTraining Gradient Boosting Regressor...")
print("  - Predicting survival time (days)")
print("  - Weighting events more than censored patients")
print("  - Using Huber loss (robust to outliers)")


# Initialize Gradient Boosting Regressor
# This model builds many small decision trees sequentially
gb_model = GradientBoostingRegressor(

    n_estimators=100,          
    # Number of trees
    # More trees = more learning capacity (but slower)

    learning_rate=0.05,        
    # Controls how much each tree contributes
    # Smaller value = slower, safer learning

    max_depth=4,               
    # Maximum depth of each tree
    # Controls how complex each tree can be

    min_samples_split=20,      
    # Minimum samples required to split a node
    # Prevents overfitting on small groups

    min_samples_leaf=10,       
    # Minimum samples allowed in a leaf node
    # Ensures predictions are based on enough patients

    subsample=0.8,             
    # Each tree uses only 80% of training data
    # Adds randomness and improves generalization

    max_features='sqrt',       
    # Each split considers only sqrt(number of features)
    # Reduces correlation between trees

    loss='huber',              
    # Huber loss is robust to extreme survival times
    # Combines advantages of MAE and MSE

    alpha=0.9,                 
    # Controls sensitivity of Huber loss to outliers

    random_state=42,           
    # Ensures reproducible results

    verbose=0                  
    # No extra output during training
)


# Train the Gradient Boosting model
# Why?
# - Model learns patterns linking patient features to survival time
# - Sample weights ensure deaths influence learning more
gb_model.fit(
    X_train,                   # Patient features
    y_train,                   # Survival time in days
    sample_weight=train_weights
)


# Confirm training completion
print("\n✓ Gradient Boosting model trained successfully!")



Training Gradient Boosting Regressor...
  - Predicting survival time (days)
  - Weighting events more than censored patients
  - Using Huber loss (robust to outliers)

✓ Gradient Boosting model trained successfully!


In [30]:
# --- H. Model Evaluation ---

# 1️⃣ Print a visual separator line
# Why?
# - Makes console output clean and organized
# - Helps clearly separate training output from evaluation output
print("\n" + "=" * 70)


# 2️⃣ Print section title
# Why?
# - Indicates that model training is finished
# - Everything printed after this relates to model performance
print("MODEL PERFORMANCE")


# 3️⃣ Print another separator line
# Why?
# - Improves readability
# - Makes logs easier to interpret during debugging or presentations
print("=" * 70)



MODEL PERFORMANCE


In [31]:
# Make predictions

# 1️⃣ Predict survival time for TRAINING patients
# Why?
# - Checks how well the model learned patterns
# - High performance here means the model fit the data
y_train_pred = gb_model.predict(X_train)


# 2️⃣ Predict survival time for TEST patients
# Why?
# - Tests model performance on unseen patients
# - This is the TRUE measure of model quality
y_test_pred = gb_model.predict(X_test)


In [32]:
# Calculate C-index
# Why?
# - Survival models are evaluated by ranking, not exact values
# - We care whether the model correctly orders patients by survival

# For survival time predictions:
# - Higher predicted time = better prognosis (lives longer)
# - So we pass predictions directly (no sign change needed)

# 1️⃣ C-index on TRAINING data
# Why?
# - Measures how well the model ranks patients it learned from
train_ci = concordance_index(
    train_df["os_time_days"],   # True survival time
    y_train_pred,               # Predicted survival time
    train_df["event"]           # Event indicator (1 = death, 0 = censored)
)


# 2️⃣ C-index on TEST data
# Why?
# - Measures how well the model ranks NEW patients
# - This is the key metric to judge real performance
test_ci = concordance_index(
    test_df["os_time_days"],
    y_test_pred,
    test_df["event"]
)


In [33]:
# Print C-index results
# Why?
# - Shows how well the model ranks patients by survival
# - Separately reports performance on training and testing data
print("\nConcordance Index (C-index):")
print(f"  Train: {train_ci:.4f}")   # Model performance on training data
print(f"  Test:  {test_ci:.4f}")    # Model performance on unseen test data


# Compare test C-index against a reference benchmark (0.72)
# Why 0.72?
# - This can represent:
#   • A baseline Cox model
#   • A previous model
#   • A clinically acceptable threshold
if test_ci > 0.72:
    
    # Calculate percentage improvement over the benchmark
    improvement = ((test_ci - 0.72) / 0.72) * 100
    
    # Print improvement message
    print(f"\n  🎉 Improvement: +{improvement:.2f}%")

else:
    
    # Calculate percentage decline from the benchmark
    decline = ((0.72 - test_ci) / 0.72) * 100
    
    # Print decline warning
    print(f"\n  ⚠️  Lower by: -{decline:.2f}%")



Concordance Index (C-index):
  Train: 0.7442
  Test:  0.7247

  🎉 Improvement: +0.66%


In [34]:
# Additional metrics for observed events

# 1️⃣ Create a mask for patients who actually died (event = 1)
# Why?
# - For these patients, we know the true survival time exactly
# - Error metrics like MAE and RMSE require exact outcomes
train_events_mask = train_df["event"] == 1
test_events_mask = test_df["event"] == 1


# 2️⃣ Check if there are any death events in the test set
# Why?
# - MAE/RMSE cannot be computed if no patient died
# - Prevents division by zero or meaningless metrics
if test_events_mask.sum() > 0:
    
    # 3️⃣ Import error metrics
    # Why?
    # - MAE measures average absolute prediction error
    # - RMSE penalizes large errors more heavily
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    
    # 4️⃣ Calculate Mean Absolute Error (MAE) for deceased patients only
    # Why?
    # - Measures average prediction mistake in days
    # - Uses only patients with known survival time
    mae = mean_absolute_error(
        y_test[test_events_mask],      # True survival time
        y_test_pred[test_events_mask]  # Predicted survival time
    )
    
    
    # 5️⃣ Calculate Root Mean Squared Error (RMSE)
    # Why?
    # - Highlights large prediction errors
    # - More sensitive to outliers than MAE
    rmse = np.sqrt(
        mean_squared_error(
            y_test[test_events_mask],
            y_test_pred[test_events_mask]
        )
    )
    
    
    # 6️⃣ Print error metrics in days and months
    # Why?
    # - Days are precise
    # - Months are easier to interpret clinically
    print(f"\nPrediction Accuracy (for observed deaths):")
    print(f"  Mean Absolute Error:  {mae:.0f} days ({mae/30:.1f} months)")
    print(f"  Root Mean Sq Error:   {rmse:.0f} days ({rmse/30:.1f} months)")



Prediction Accuracy (for observed deaths):
  Mean Absolute Error:  499 days (16.6 months)
  Root Mean Sq Error:   703 days (23.4 months)


In [35]:
# --- I. Feature Importance ---

# 1️⃣ Print section header
# Why?
# - Clearly separates feature importance output
# - Improves readability of results
print("\n" + "=" * 70)
print("FEATURE IMPORTANCE")
print("=" * 70)


# 2️⃣ Create a table of feature importance scores
# Why?
# - Tree-based models can measure how much each feature
#   contributes to reducing prediction error
feature_importance = pd.DataFrame({

    'Feature': X_train.columns,  
    # Name of each input feature used by the model

    'Importance': gb_model.feature_importances_
    # Numerical score showing how influential each feature is

}).sort_values(
    'Importance',
    ascending=False   # Sort from most important to least important
)


# 3️⃣ Print heading for top features
# Why?
# - Focuses attention on the most influential variables
print("\nTop 10 Most Important Features:")
print("(These features have the strongest impact on survival predictions)\n")


# 4️⃣ Print top 10 features with a visual bar
# Why?
# - Makes importance easier to interpret visually
# - Bars help compare features at a glance
for idx, row in feature_importance.head(10).iterrows():
    
    # Create a simple bar using block characters
    bar = "█" * int(row['Importance'] * 100)
    
    # Print feature name, importance score, and bar
    print(f"  {row['Feature']:<35} {row['Importance']:.4f} {bar}")



FEATURE IMPORTANCE

Top 10 Most Important Features:
(These features have the strongest impact on survival predictions)

  hassurgery                          0.2980 █████████████████████████████
  age                                 0.1689 ████████████████
  groupstage_Stage IA                 0.1218 ████████████
  groupstage_Stage IV                 0.1054 ██████████
  race_White                          0.0499 ████
  groupstage_Stage I                  0.0368 ███
  groupstage_Stage IB                 0.0307 ███
  smokingstatus_No history of smoking 0.0290 ██
  ismetastatic                        0.0231 ██
  groupstage_Stage IVB                0.0200 ██


In [36]:
# --- J. Risk Stratification ---

# 1️⃣ Print section header
# Why?
# - Clearly marks the start of risk stratification
# - Separates this analysis from previous evaluation steps
print("\n" + "=" * 70)
print("RISK STRATIFICATION")
print("=" * 70)


# 2️⃣ Create a copy of the test dataset
# Why?
# - We want to attach predictions without modifying original test data
# - Keeps raw test data safe for future analysis
test_results = test_df.copy()


# 3️⃣ Add predicted survival time to test data
# Why?
# - Allows direct comparison between predicted and actual survival
# - Required for grouping patients by predicted risk
test_results['predicted_survival'] = y_test_pred


# 4️⃣ Create a risk score
# Why?
# - Risk is the inverse of survival time
# - Shorter predicted survival = higher risk
# - Negative sign converts survival prediction into risk ranking
test_results['risk_score'] = -y_test_pred  # Lower survival → higher risk



RISK STRATIFICATION


In [37]:
# Create risk groups based on predicted survival

# 1️⃣ Divide patients into 3 risk groups using predicted survival
# Why?
# - Doctors prefer categories (High / Medium / Low risk)
# - q=3 splits patients into equal-sized groups (tertiles)
test_results['risk_group'] = pd.qcut(
    test_results['predicted_survival'],   # Predicted survival time
    q=3,                                  # Number of groups
    labels=['High Risk', 'Medium Risk', 'Low Risk']
    # Lower survival → High Risk
)


# 2️⃣ Print heading for risk group evaluation
# Why?
# - Clearly separates results by risk category
print("\nRisk Group Performance:")


# 3️⃣ Loop through each risk group
# Why?
# - Allows separate evaluation of High, Medium, and Low risk patients
for group in ['High Risk', 'Medium Risk', 'Low Risk']:
    
    # 4️⃣ Select patients belonging to the current risk group
    group_data = test_results[test_results['risk_group'] == group]
    
    
    # 5️⃣ Count number of patients in this group
    # Why?
    # - Ensures groups are balanced
    n = len(group_data)
    
    
    # 6️⃣ Calculate median predicted survival
    # Why?
    # - Median is robust to extreme survival values
    pred_median = group_data['predicted_survival'].median()
    
    
    # 7️⃣ Calculate median actual survival
    # Why?
    # - Allows comparison between prediction and reality
    actual_median = group_data['os_time_days'].median()
    
    
    # 8️⃣ Calculate death rate in the group
    # Why?
    # - High-risk group should have higher death rate
    # - Validates that stratification is meaningful
    event_rate = group_data['event'].mean() * 100
    
    
    # 9️⃣ Print group-level results
    print(f"\n  {group} (n={n}):")
    print(f"    Predicted Median Survival: {pred_median:.0f} days ({pred_median/30:.1f} months)")
    print(f"    Actual Median Survival:    {actual_median:.0f} days ({actual_median/30:.1f} months)")
    print(f"    Death Event Rate:          {event_rate:.1f}%")



Risk Group Performance:

  High Risk (n=86):
    Predicted Median Survival: 388 days (12.9 months)
    Actual Median Survival:    257 days (8.6 months)
    Death Event Rate:          72.1%

  Medium Risk (n=86):
    Predicted Median Survival: 594 days (19.8 months)
    Actual Median Survival:    513 days (17.1 months)
    Death Event Rate:          57.0%

  Low Risk (n=86):
    Predicted Median Survival: 1234 days (41.1 months)
    Actual Median Survival:    1238 days (41.3 months)
    Death Event Rate:          36.0%


In [38]:
# --- K. Example Predictions ---

# 1️⃣ Print section header
# Why?
# - Clearly marks the start of example predictions
# - Makes output easier to read
print("\n" + "=" * 70)
print("EXAMPLE PREDICTIONS")
print("=" * 70)


# 2️⃣ Print explanation text
# Why?
# - Tells the reader what will be shown next
print("\nFirst 5 patients in test set:\n")


# 3️⃣ Convert true survival values to NumPy array
# Why?
# - Makes indexing easier inside the loop
y_test_values = y_test.values


# 4️⃣ Loop through first 5 patients in test data
# Why?
# - Shows real examples instead of abstract metrics
# - Limits output so it stays readable
for i in range(min(5, len(test_df))):

    # 5️⃣ Get predicted survival time for patient i
    # Why?
    # - This is what the model predicts
    pred_survival = y_test_pred[i]


    # 6️⃣ Get actual survival time for patient i
    # Why?
    # - This is the ground truth (if known)
    actual_survival = y_test_values[i]


    # 7️⃣ Check whether the patient died or is censored
    # Why?
    # - Helps interpret the prediction error
    is_event = test_df.iloc[i]['event']
    status = "Died" if is_event else "Censored"


    # 8️⃣ Extract key patient features
    # Why?
    # - Allows understanding WHY the model predicted this value
    age = test_df.iloc[i]['age']
    metastatic = test_df.iloc[i]['ismetastatic']
    surgery = test_df.iloc[i]['hassurgery']
    ecog = test_df.iloc[i]['ecogvalue']


    # 9️⃣ Print patient number
    print(f"Patient {i+1}:")


    # 🔟 Print predicted survival
    # Why?
    # - Shows model output in days and months
    print(f"  Predicted Survival: {pred_survival:.0f} days ({pred_survival/30:.1f} months)")


    # 1️⃣1️⃣ Print actual survival and status
    # Why?
    # - Shows how close the model was
    # - Indicates if survival time is exact or censored
    print(f"  Actual Survival:    {actual_survival:.0f} days ({actual_survival/30:.1f} months) [{status}]")


    # 1️⃣2️⃣ Print patient clinical profile
    # Why?
    # - Helps interpret the prediction
    # - Matches how doctors think
    print(
        f"  Age: {age:.0f}, "
        f"ECOG: {ecog:.0f}, "
        f"Metastatic: {'Yes' if metastatic else 'No'}, "
        f"Surgery: {'Yes' if surgery else 'No'}"
    )


    # 1️⃣3️⃣ Print prediction error ONLY if patient died
    # Why?
    # - Exact error can be computed only for death events
    if is_event:
        error = abs(pred_survival - actual_survival)
        print(f"  Prediction Error:   {error:.0f} days ({error/30:.1f} months)")


    # 1️⃣4️⃣ Blank line for readability
    print()



EXAMPLE PREDICTIONS

First 5 patients in test set:

Patient 1:
  Predicted Survival: 1828 days (60.9 months)
  Actual Survival:    805 days (26.8 months) [Censored]
  Age: 56, ECOG: 1, Metastatic: No, Surgery: Yes

Patient 2:
  Predicted Survival: 260 days (8.7 months)
  Actual Survival:    89 days (3.0 months) [Died]
  Age: 75, ECOG: 2, Metastatic: Yes, Surgery: No
  Prediction Error:   171 days (5.7 months)

Patient 3:
  Predicted Survival: 1228 days (40.9 months)
  Actual Survival:    2695 days (89.8 months) [Died]
  Age: 76, ECOG: 1, Metastatic: No, Surgery: Yes
  Prediction Error:   1467 days (48.9 months)

Patient 4:
  Predicted Survival: 1198 days (39.9 months)
  Actual Survival:    489 days (16.3 months) [Censored]
  Age: 51, ECOG: 1, Metastatic: Yes, Surgery: Yes

Patient 5:
  Predicted Survival: 1090 days (36.3 months)
  Actual Survival:    2762 days (92.1 months) [Censored]
  Age: 71, ECOG: 1, Metastatic: No, Surgery: Yes



In [41]:
# --- L. Final Model Summary ---
# Purpose:
# - Report final model performance
# - Justify why Gradient Boosting was used
# - Show interpretability and limitations

print("=" * 60)
print("FINAL MODEL SUMMARY")
print("=" * 60)

# ✅ 1. Core performance metric (MOST IMPORTANT)
# Why?
# - C-index is the standard metric for survival models
# - Test score reflects real-world performance
print(f"""
Model Performance:
-----------------
Gradient Boosting (Test C-index): {test_ci:.4f}
""")

# ✅ 2. Why Gradient Boosting instead of Cox
# Why?
# - Explains modeling choice clearly
# - Common interview question
print("""
Why Gradient Boosting:
---------------------
✓ No proportional hazards assumption
✓ Captures non-linear relationships automatically
✓ Learns feature interactions (e.g., Surgery × Stage)
✓ Robust to extreme survival times (Huber loss)
✓ Allows higher weight to death events than censored cases
""")

# ✅ 3. Model interpretability (VERY IMPORTANT)
# Why?
# - Shows the model aligns with clinical intuition
# - Builds trust in predictions
print("Top 3 Most Important Features:")
for idx, row in feature_importance.head(3).iterrows():
    print(f"  {idx+1}. {row['Feature']} (importance = {row['Importance']:.3f})")

# ✅ 4. Limitations (DO NOT SKIP – shows maturity)
# Why?
# - Prevents overclaiming
# - Required in research and interviews
print("""
Limitations:
------------
• No native survival-specific loss function
• Censored observations handled approximately
• Requires hyperparameter tuning for optimal results
• Less interpretable than Cox regression coefficients
""")



FINAL MODEL SUMMARY

Model Performance:
-----------------
Gradient Boosting (Test C-index): 0.7247


Why Gradient Boosting:
---------------------
✓ No proportional hazards assumption
✓ Captures non-linear relationships automatically
✓ Learns feature interactions (e.g., Surgery × Stage)
✓ Robust to extreme survival times (Huber loss)
✓ Allows higher weight to death events than censored cases

Top 3 Most Important Features:
  4. hassurgery (importance = 0.298)
  1. age (importance = 0.169)
  11. groupstage_Stage IA (importance = 0.122)

Limitations:
------------
• No native survival-specific loss function
• Censored observations handled approximately
• Requires hyperparameter tuning for optimal results
• Less interpretable than Cox regression coefficients

